## Step 1: Pre-Preprocessing

In [20]:
%load_ext autoreload
%autoreload 2

# import preprocessing code
from src.preprocess import PreProcessor, df_to_train_set, df_to_test_set

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# save paths to the available datasets
from typing import NamedTuple, List

class Dataset(NamedTuple):
    """
    Interface for accessing data folders.
    """
    title: str
    preprocessed_folder: str
    raw_folders: List[str]

SAMPLE_DATA = Dataset(
    title = "sample_data",
    preprocessed_folder = "../de-ID_data/preprocessed/sample_data/",
    raw_folders = ["docs/Track1-de-indentification/PHI/"]
)

GOLD_1 = Dataset(
    title = "gold_1",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_1/",
    raw_folders = ["../de-ID_data/raw/training-PHI-Gold-Set1/"]
)

GOLD_FULL = Dataset(
    title = "gold_full",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_full/",
    raw_folders = ["../de-ID_data/raw/training-PHI-Gold-Set1/","../data/raw/training-PHI-Gold-Set2/"]
)

GOLD_TEST = Dataset(
    title = "gold_test",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_test/",
    raw_folders = ["../de-ID_data/raw/testing-PHI-Gold-fixed/"]
)

DATASETS = [SAMPLE_DATA,GOLD_1,GOLD_FULL, GOLD_TEST]

## Step 2: Preprocessing

In [3]:
# pick dataset and define loading boolean
train_data = DATASETS[2]
# train_data = DATASETS[0]
test_data = DATASETS[3]
isLoading = True

In [4]:
# attach data to PreProcessor object.
pp = PreProcessor(train_data.title)
if isLoading:
    X_train,y_train,df_train = pp.get_data(train_data.preprocessed_folder,isLoading = isLoading)
else:
    X_train,y_train,df_train = pp.get_data(train_data.raw_folders,isLoading = isLoading)
print("max length: ",pp.max_len)

Loading preprocessed data...
100%|██████████| 31535/31535 [00:05<00:00, 6253.60it/s]
Preprocessing complete.
max length:  1567


In [5]:
# data exploration
df_train.head()

,Unnamed: 0,docid,sentence,token,token_id,label,label_id,characters
0,0,220-01,0,Record,10503,O,1,"(3, 9)"
1,1,220-01,0,date,23254,O,1,"(10, 14)"
2,2,220-01,0,:,20619,O,1,"(14, 15)"
3,3,220-01,0,2067,18377,B-DATE,42,"(16, 20)"
4,4,220-01,0,-,24073,I-DATE,18,"(20, 21)"


In [6]:
# load test set
isLoadingTest = True
if isLoadingTest:
    X_test,y_test,df_test = pp.create_test_set(test_data.preprocessed_folder,isLoadingTest,test_data.title)
else:
    X_test,y_test,df_test = pp.create_test_set(test_data.raw_folders,isLoadingTest,test_data.title)

Loading preprocessed test data...
100%|██████████| 21670/21670 [00:02<00:00, 7599.57it/s]


In [7]:
# test data exploration
df_test.head()

,Unnamed: 0,docid,sentence,token,token_id,label,label_id,characters,original_token
0,0,110-01,0,Record,10503,O,1,"(3, 9)",Record
1,1,110-01,0,date,23254,O,1,"(10, 14)",date
2,2,110-01,0,:,20619,O,1,"(14, 15)",:
3,3,110-01,0,2069,9322,B-DATE,42,"(16, 20)",2069
4,4,110-01,0,-,24073,I-DATE,18,"(20, 21)",-


In [77]:
# import model stuff
from src.models.baseline import BaselineModel
from src.models.bilstm import BiLSTM
from src.models.bilstm_crf import BiLSTM_CRF
from src.models.transformer import Transformer
from src.models.transformer_crf import Transformer_CRF
from src.models.transformer_bilstm import TransformerBiLSTM
from src.models.bilstm_chars import BiLSTM_Chars
from src.models.bilstm_chars_crf import BiLSTM_Chars_CRF
from pipeline.visualization import sample_output
from pipeline.train import train
from random import randint
from sklearn.utils import shuffle
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from src.converter import get_label_positions, bio_to_i2d2
import xml.etree.ElementTree as ET
from typing import NamedTuple, List

In [9]:
# check if GPU is available
assert tf.test.is_built_with_cuda()
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num GPUs:", len(physical_devices)) 
tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)

Num GPUs Available:  1
Num GPUs: 1


report_tensor_allocations_upon_oom: true

In [78]:
# build model
# model = BaselineModel(pp.vocab_size,pp.tag_size,pp.max_len)
# model = BiLSTM(pp.vocab_size,pp.tag_size,pp.max_len)
# model = BiLSTM_CRF(pp.vocab_size,pp.tag_size,pp.max_len)
# model = Transformer(pp.vocab_size,pp.tag_size,pp.max_len)
# model = Transformer_CRF(pp.vocab_size, pp.tag_size, pp.max_len)
# model = TransformerBiLSTM(pp.vocab_size, pp.tag_size, pp.max_len)
# model = BiLSTM_Chars(pp.vocab_size, pp.tag_size, pp.max_len,pp.idx2word)
model = BiLSTM_Chars_CRF(pp.vocab_size, pp.tag_size, pp.max_len,pp.idx2word)

In [79]:
# configure checkpoints and checkpoint manager
checkpoint_dir = 'models/checkpoints/' + train_data.title + '/' + model.title + '/' 
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=10)

In [80]:
# restore checkpoint
checkpoint.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))

In [82]:
# train
print("Training ",model.title)
losses = train(model,X_train,y_train,batch_size = 32, epochs=10, lr = 0.0005, sample_interval=10,manager=manager,pp=pp)

Training  bi-lstm-chars-crf
--------- EPOCH  0 -----------
Epoch: 0, Batch: 0, Loss: 104.194000
Epoch: 0, Batch: 5, Loss: 45.384926


KeyboardInterrupt: 

In [13]:
# sample a random train output
sample_output(model,X_train,y_train, pp = pp,rand_idx=None)

Sentence #:  29377
Word            Pred : (True)
No             :O     (O)
polis          :O     (O)
Pt             :O     (O)
did            :O     (O)
not            :O     (O)
bring          :O     (O)
BS             :O     (O)
chart          :O     (O)
.              :O     (O)


In [27]:
X_test, y_test, X_test_words = df_to_test_set(df_test)

100%|██████████| 21670/21670 [00:02<00:00, 7689.98it/s]


In [74]:
# sample a random test output
# print(X_test_words[0][:30])
sample_output(model,X_test,y_test, pp = pp,rand_idx=None, train_words = X_test_words)

Sentence #:  5613
['Fe' 'So4' '325' ... 'PAD' 'PAD' 'PAD']
Word            Pred : (True)
Fe             :O     (O)
UNK            :O     (O)
325            :O     (O)
mg             :O     (O)
QD             :O     (O)
5              :O     (O)
.              :O     (O)


In [52]:
# test model
from pipeline.test import test_to_i2d2

test_to_i2d2(model,df_test, pp, checkpoint, manager)

c:\Users\abdul\Desktop\de-identification
Loading checkpoint...
100%|██████████| 494/494 [03:33<00:00,  2.32it/s]
